<a href="https://colab.research.google.com/github/Isaac-Gathere/Isaac-Gathere/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install necessary packages using pip
!pip install keras numpy wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import io
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, TimeDistributed
from keras.utils import to_categorical as keras_to_categorical
from keras.utils import pad_sequences
import nltk
import sys
import string

In [ ]:
def load_corpus(path):
    # Check if the path is a directory.
    if not os.path.isdir(path):
        sys.exit("Input path is not a directory")
    
    # Define lists to hold sentences and their corresponding tags.
    sentences, tags = [], []

    # Iterate through each file in the directory.
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        with open(file_path, "r") as f:
            sentence, tag = [], []
            for line in f:
                # Split each line into a word and its corresponding tag.
                parts = line.strip().split("\t")
                if len(parts) == 2:
                    word, tag_ = parts
                    sentence.append(word)
                    tag.append(tag_)
            sentences.append(sentence)
            tags.append(tag)
    return sentences, tags

# Test the function here:
path = "/content/drive/MyDrive/NLP/modified_brown"
data = load_corpus(path)

In [ ]:
# Creates the dataset with train_X (words) and train_y (tag).
def create_dataset(sentences):
    # Defines the relevant lists.
    train_X, train_y = list(), list()

    # Iterate over the sentences and tags, and create the dataset.
    for sentence in sentences:
        words, tags = [], []
        for word_tag in sentence:
            word, tag = word_tag.split("_")
            words.append(word)
            tags.append(tag)
        train_X.append(words)
        train_y.append(tags)

    return train_X, train_y

# Test the function here.
train_X, train_y = create_dataset(data[0])


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences as pad

# Pad the sequences with 0s to the max length.
def pad_sequences(train_X, train_y):
    # Use MAX_LENGTH to record length of longest sequence 
    MAX_LENGTH = max(len(sentence) for sentence in train_X)
    
    # Pad the sequences with 0s to the max length.
    train_X = pad(train_X, maxlen=MAX_LENGTH, padding="post", value=0)
    train_y = pad(train_y, maxlen=MAX_LENGTH, padding="post", value=0)

    return train_X, train_y, MAX_LENGTH

# Test the function
train_X, train_y, MAX_LENGTH = pad_sequences(train_X, train_y)


In [ ]:

from keras.layers import InputLayer, Activation
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding
from keras.optimizers import Adam



In [ ]:
def define_model(MAX_LENGTH, len_word2idx, len_tag2idx):
    # Define a sequential Keras model.
    model = Sequential()

    # Add an embedding layer to convert words to vectors.
    model.add(Embedding(input_dim=len_word2idx, output_dim=100, input_length=MAX_LENGTH))

    # Add a bidirectional LSTM layer to extract features from the sequences.
    model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))

    # Add a time distributed dense layer to apply a Dense layer to each output element of the bidirectional LSTM.
    model.add(TimeDistributed(Dense(50, activation="relu")))

    # Add a fully connected layer to compute the probability of each tag.
    model.add(Dense(len_tag2idx, activation="softmax"))

    # Define an optimizer and compile the model.
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=0.0)
    model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])

    print(model.summary())
    return model

In [ ]:
# Returns the one-hot encoding of the sequence.
def to_categorical(train_y, num_tags=11):
    # Convert the sequence of tags to one-hot encoding
    categorical_y = np.zeros((train_y.shape[0], train_y.shape[1], num_tags))
    for i, sentence in enumerate(train_y):
        for j, tag in enumerate(sentence):
            categorical_y[i, j, tag] = 1
    return categorical_y


In [ ]:
def train(model, train_X, train_y):
    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = [tf.keras.metrics.CategoricalAccuracy()]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    model.fit(train_X, train_y, epochs=40)

    return model
    model = train(model, train_X, train_y)

In [ ]:
def test(model, sentence):

    # Tokenize the sentence and convert it to a sequence of word indices
    test_sequence = []
    for word in sentence:
        if word.lower() in word2idx:
            test_sequence.append(word2idx[word.lower()])
        else:
            # If the word is not in the vocabulary, use the <UNK> token
            test_sequence.append(word2idx['<UNK>'])
    test_sequence = np.array(test_sequence).reshape(1, -1)

    # Use the model to predict the tags for each word in the sentence
    predicted_tags = model.predict(test_sequence)

    # Convert the predicted tags to their corresponding tag names
    predicted_tags = [idx2tag[idx] for idx in np.argmax(predicted_tags, axis=-1).squeeze()]

    # Print the sentence with the predicted tags
    for word, tag in zip(sentence, predicted_tags):
        print(f"{word}/{tag}", end=" ")
    print("\n")
    test(model, testString1)
    test(model, testString2.split())



In [ ]:
sentence = ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
print(list(zip(sentence)))

[('The',), ('quick',), ('brown',), ('fox',), ('jumps',), ('over',), ('the',), ('lazy',), ('dog',)]
